In [1]:
import sys
sys.executable

'/usr/local/opt/python/bin/python3.7'

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial import KDTree
from scipy.stats import uniform, expon, poisson, describe
import math

import matplotlib.pyplot as plt

from copy import deepcopy
from collections import defaultdict

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import pickle

In [3]:
# this notebook is to fix heuristic 1 and develop heuristic 2

In [18]:
filename = 'slurm scripts/data/08-14-20/path_points/'

points = []
distances = []
lengths_based = []
counts = []

T_n_indices = defaultdict(list)

for i in range(1,501):
    # something weird happened, if i is 0 mod 50 the file doesn't exist...
    #if i % 50 == 0:
    #    continue
    
    f = open(filename+str(i)+'-dim2-n5000000_pathpoints.pkl', "rb")
    pathpoints = pickle.load(f)
    points.append(pathpoints)
    s_n = 0
    for j in range(len(pathpoints)-2):
        s_n += np.linalg.norm(pathpoints[j+1] - pathpoints[j])
    lengths_based.append(s_n)
    distances.append(s_n + np.linalg.norm(pathpoints[-1] - pathpoints[-2]))
    counts.append(len(pathpoints)-2)
    T_n_indices[len(pathpoints)-2].append(i)
    
    if pathpoints[-1][0] != 0.9 or pathpoints[-1][1] != 0.9:
        print(i)

        
distances = np.array(distances)
lengths_based = np.array(lengths_based)
counts_based = np.array(counts)

In [4]:
n = 5000000
D = 2

ell = 0.8 * np.sqrt(2)

x_init = np.array([0, 0])
x_goal = np.array([ell, 0])

In [5]:
# r_n function
def r(n, D):
    return (n ** (-1/(2*D))) / 5
    # designed for n = 5 * 10^6

In [8]:
th_max = math.pi * 0.1

In [9]:
lambda_r = n * (math.pi * (r(n, D) ** 2))

In [10]:
print(lambda_r)
print(th_max)
print(lambda_r * th_max)

280.9925892416291
0.3141592653589793
88.27642540746763


In [6]:
# Creates random sample of n points (not including init and goal) -- poisson functionality added
def SampleFree(n, d, x_init, x_goal, distr='unif'):
    assert distr in {'unif', 'pois'}, "distr parameter must be one of the following: 'unif', 'pois'"
    
    # start with init point (index 0)
    V = [x_init]
    
    if distr is 'pois':
        num_points = np.random.poisson(lam=n)
    else:
        num_points = n
    
    for i in range(num_points):
        point = np.random.uniform(size=d)
        if (point[0]-0.5)**2 - 1.997 * (point[0]-0.5) * (point[1]-0.5) + (point[1]-0.5)**2 > 1/1800:
            continue
        V.append(point)
        
    # add the goal point (index n + 1)
    V.append(x_goal)
    
    return V

In [7]:
# Creates adjacency list, no need for also storing their distances
def Near(V, r_n):
    
    # Create a KD Tree first
    KDT = KDTree(data=V)
    
    # Create the KD Tree to search against
    search_against = KDTree(data=V)

    # run query_ball_tree; returns list of lists
    results = KDT.query_ball_tree(other=search_against, r=r_n)
    
    # for vertex indexed by i, results[i] contains all indices j such that dist(v[i],v[j]) < r
    edges = [None] * len(V)
    for i in range(len(V)):
        edges[i] = []
        for j in results[i]:
            if i != j:
                edges[i].append(j)

In [7]:
# graph heuristic where in a sector of width theta_max about the beeline we take the point with max R
def path_algorithm_2(V, E):
    # V is the vertex set, E is the output of the KDT function (both numpy arrays)
    x_init = V[0]
    x_goal = V[-1]
    
    # tracks where we've been (array of indices)
    piece = 0
    visited = [0]
    distance = 0
    
    # boolean indicator of something going wrong, e.g. no further point or repeat node
    failure = False
    #print((failure is False) and (piece != len(V)-1))
    
    while (failure is False) and (piece != len(V)-1):
        candidates = np.take(V, np.array(E[piece]), axis=0)
        angles = []
        for p in candidates:
            vect_1 = p - V[piece]
            vect_2 = x_goal - V[piece]
            angle = math.atan2( vect_1[0]*vect_2[1] - vect_1[1]*vect_2[0], vect_1[0]*vect_2[0] + vect_1[1]*vect_2[1])
            angles.append(np.abs(angle))             # absolute angle in radians
        next_piece = E[piece][np.array(angles).argmin()]
        if next_piece in visited:
            #print('failed')
            #print(next_piece)
            #print(visited)
            failure = True            # means we have already been to this node
        else:
            distance += np.linalg.norm(V[next_piece] - V[piece])
            visited.append(next_piece)
            piece = next_piece
            
    path_nodes = []
    for v in visited:
        path_nodes.append(V[v])
    
    if failure:
        return path_nodes, float('inf')
    
    else:
        return path_nodes, distance